In [ ]:
import datetime
import csv

from gensim import corpora
from gensim import models
from pprint import pprint  # pretty-printer
from gensim import similarities

import re

from nltk.corpus import stopwords
from nltk import PorterStemmer

In [ ]:
init_t: datetime = datetime.datetime.now()  # init the time for the execution time calculation

In [ ]:
news_file = "news.csv"

CSV extraction #

In [ ]:
all_news = []
descriptions = []
#food_drink_news = []

In [ ]:
with open(news_file, 'r', newline='', encoding='utf-8') as csv_file:
    reader_csv = csv.reader(csv_file)
    for line in reader_csv:
        all_news.append(line)
        descriptions.append(line[3])

        #if ")Food & Drink" in line[2]:
            #food_drink_news.append(line)

In [ ]:
# delete the first row 'description' name
descriptions.pop(0)
descriptions

In [ ]:
porter = PorterStemmer()

In [ ]:
# remove common words and tokenize
stoplist = stopwords.words('english')
texts = [
    [porter.stem(word) for word in document.lower().split() if word not in stoplist]
    for document in descriptions
]

In [ ]:
print("Tokens of each document:")
texts

In [ ]:
# create mapping keyword-id
dictionary = corpora.Dictionary(texts)

In [ ]:
print()
print("Mapping keyword-id:")
pprint(dictionary.token2id)

create the vector for each doc

In [ ]:
model_bow = [dictionary.doc2bow(text) for text in texts]

create tfidf model

In [ ]:
tfidf = models.TfidfModel(model_bow)
tfidf_vectors = tfidf[model_bow]

In [ ]:
id2token = dict(dictionary.items())

In [ ]:
def convert(match):
    return dictionary.id2token[int(match.group(0)[0:-1])]

In [ ]:
print()
print("Vectors for documents (the positions with zeros are not shown):")
for doc in tfidf_vectors:
    print(re.sub("[0-9]+,", convert, str(doc)))

In [ ]:
matrix_tfidf = similarities.MatrixSimilarity(tfidf_vectors)  # this matrix will be necessary to calculate similarity between documents

In [ ]:
end_creation_model_t: datetime = datetime.datetime.now()  # just after the calculation of the matrix similarity -> time function

In [ ]:
print()
print("Matrix similarities")
print(matrix_tfidf)

obtain tfidf vector for the following doc

In [ ]:
doc = "trees graph human"
doc_s = [porter.stem(word) for word in doc.lower().split() if word not in stoplist]

In [ ]:
vec_bow = dictionary.doc2bow(doc_s)
vec_tfidf = tfidf[vec_bow]

calculate similarities between doc and each doc of texts using tfidf vectors and cosine

In [ ]:
sims = matrix_tfidf[vec_tfidf]  # sims is a list a similarities

sort similarities in descending order

In [ ]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])

In [ ]:
print()
print("Given the doc: " + doc)
print("whose tfidf vector is: " + str(vec_tfidf))
print()
print("The Similarities between this doc and the documents of the corpus are:")
for doc_position, doc_score in sims:
    print(doc_score, descriptions[doc_position])

In [ ]:
end_t: datetime = datetime.datetime.now()  # to mark the end of the program

get execution time

In [ ]:
elapsed_time_model_creation: datetime = end_creation_model_t - init_t
elapsed_time_comparison: datetime = end_t - end_creation_model_t
print()
print('Execution time model:', elapsed_time_model_creation, 'seconds')
print('Execution time comparison:', elapsed_time_comparison, 'seconds')